<a href="https://colab.research.google.com/github/hedraatia-dev/functions_data_analysis_with_HedraAtia/blob/main/hedra_group_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

توثيق منصة التحليل التفاعلية   

hedra_group

الإصدار: 1.0

تاريخ الإنشاء: 4 سبتمبر 2025

المؤلفون: Hedra Atia و Manus

In [1]:
!pip install openpyxl
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.9 MB/s eta 0:00:00


الوظيفة الأساسية (hedra_group_final):

 هي قلب الأداة.

تم تصميمها لتكون مستقلة وقابلة لإعادة الاستخدام.

تستقبل DataFrame ومجموعة من الخيارات،

 وتعيد كائن Styler منسقًا

In [2]:
# --- الخطوة 1: تثبيت المكتبات (إذا لم تكن مثبتة) ---
# !pip install ipywidgets
# !pip install openpyxl

# --- الخطوة 2: استيراد المكتبات المطلوبة ---

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets as widgets
from IPython.display import display, clear_output
import io


# إعدادات الشكل الافتراضي للرسوم
plt.style.use("ggplot")
sns.set_theme()
sns.set_palette("pastel")

# --- الخطوة 3: وظيفة hedra_group_final (لا تحتاج لتغيير) ---
def hedra_group_final(df, by, cols, agg_func='sum',
                      grand_total=True, total_color='yellow',
                      gradient=True, gradient_cmap='YlGnBu',
                      export_excel=False, excel_path='hedra_report.xlsx'):

    if df is None:
        print("الرجاء رفع ملف بيانات أولاً.")
        return
    if not cols or not by:
        print("الرجاء اختيار عمود تجميع وعمود رقمي واحد على الأقل.")
        return

    grouped = df.groupby(by)[cols].agg(agg_func).reset_index()
    if grand_total:
        total = df[cols].agg(agg_func)
        total_row = {col: total.get(col, '') for col in grouped.columns}
        by_cols = by if isinstance(by, list) else [by]
        total_row[by_cols[0]] = "الإجمالي"
        for col in by_cols[1:]: total_row[col] = ''
        grouped = pd.concat([grouped, pd.DataFrame([total_row])], ignore_index=True)
    styler = grouped.style
    numeric_cols = [col for col in cols if col in grouped.columns]
    styler.format('{:,.2f}', subset=numeric_cols)
    if gradient:
        styler.background_gradient(cmap=gradient_cmap, subset=numeric_cols)
    if grand_total:
        by_col_name = (by if isinstance(by, list) else [by])[0]
        total_row_index = grouped[grouped[by_col_name] == 'الإجمالي'].index
        styler.set_properties(subset=pd.IndexSlice[total_row_index, :], **{'background-color': total_color, 'color': 'black', 'font-weight': 'bold'})
    styler.hide(axis='index')
    if export_excel:
        styler.to_excel(excel_path, engine='openpyxl', index=False)
        print(f"تم تصدير التقرير بنجاح إلى: {excel_path}")
    return styler

# --- الخطوة 4: إنشاء عناصر الواجهة ---
current_df = None

uploader = widgets.FileUpload(
    accept='.xlsx,.csv',
    multiple=False,
    description='ارفع ملف البيانات'
)

group_by_widget = widgets.SelectMultiple(description='التجميع حسب:')
columns_widget = widgets.SelectMultiple(description='الأعمدة الرقمية:')
agg_func_widget = widgets.Dropdown(options=['sum', 'mean', 'max', 'min'], value='sum', description='الدالة:')
grand_total_widget = widgets.Checkbox(value=True, description='إظهار الإجمالي')
total_color_widget = widgets.ColorPicker(value='yellow', description='لون الإجمالي:')
export_widget = widgets.Checkbox(value=False, description='تصدير إلى Excel')
run_button = widgets.Button(description="إنشاء التقرير", button_style='success', disabled=True)
output_container = widgets.Output()
status_label = widgets.Label(value="في انتظار رفع الملف...")

# --- الخطوة 5: دالة لمعالجة الملف المرفوع (تصحيح جديد) ---
def on_file_upload(change):
    global current_df

    # التحقق من وجود قيمة في أداة الرفع
    if not uploader.value:
        run_button.disabled = True
        status_label.value = "في انتظار رفع ملف..."
        current_df = None
        with output_container:
            clear_output()
        return

    try:
        # uploader.value هو قاموس، المفتاح هو اسم الملف
        uploaded_filename = list(uploader.value.keys())[0]
        uploaded_file_content = uploader.value[uploaded_filename]['content']

        if uploaded_filename.endswith('.csv'):
            current_df = pd.read_csv(io.BytesIO(uploaded_file_content))
        elif uploaded_filename.endswith('.xlsx'):
            current_df = pd.read_excel(io.BytesIO(uploaded_file_content))
        else:
            status_label.value = "صيغة الملف غير مدعومة."
            return

        status_label.value = f"تم تحميل الملف بنجاح: {uploaded_filename}"

        categorical_cols = current_df.select_dtypes(include=['object', 'category']).columns.tolist()
        numerical_cols = current_df.select_dtypes(include=np.number).columns.tolist()
        group_by_widget.options = categorical_cols
        columns_widget.options = numerical_cols

        run_button.disabled = False

    except Exception as e:
        status_label.value = f"خطأ في قراءة الملف: {e}"
        run_button.disabled = True

# ربط الدالة بزر الرفع
uploader.observe(on_file_upload, names='value')

# --- الخطوة 6: دالة التحليل ---
def run_analysis(b):
    with output_container:
        clear_output(wait=True)
        result = hedra_group_final(
            df=current_df,
            by=list(group_by_widget.value),
            cols=list(columns_widget.value),
            agg_func=agg_func_widget.value,
            grand_total=grand_total_widget.value,
            total_color=total_color_widget.value,
            export_excel=export_widget.value
        )
        if result:
            display(result)

run_button.on_click(run_analysis)

# --- الخطوة 7: عرض الواجهة النهائية ---
upload_box = widgets.VBox([uploader, status_label])
analysis_box = widgets.VBox([
    widgets.HBox([group_by_widget, columns_widget]),
    agg_func_widget,
    widgets.HBox([grand_total_widget, total_color_widget]),
    export_widget,
    run_button
])

print("منصة التحليل التفاعلية النهائية")
display(widgets.VBox([upload_box, analysis_box, output_container]))


منصة التحليل التفاعلية النهائية


بالتأكيد!

هذه هي الخطوة التي تميز الأداة الجيدة عن الأداة الرائعة.

كتابة شرح مفصل (Documentation)

سيجعل كل العمل الذي قمنا به قابلاً للاستخدام

 من قبل الآخرين بسهولة،

  ويضمن أنك أنت نفسك يمكنك العودة إليه في المستقبل وتذكر كل تفاصيله.

 بكتابة شرح شامل ومفصل،

 منظم بطريقة احترافية،

 يغطي كل جوانب منصة التحليل التفاعلية التي بنيناها.


توثيق منصة التحليل التفاعلية

hedra_group

الإصدار: 1.0

تاريخ الإنشاء: 4 سبتمبر 2025

المؤلفون: Hedra Atia و Manus

1. نظرة عامة

منصة التحليل التفاعلية hedra_group

 هي أداة قوية مبنية بلغة بايثون

 باستخدام مكتبات pandas و ipywidgets

 . تتيح هذه الأداة للمستخدمين رفع ملفات البيانات

 (Excel أو CSV) وإجراء عمليات تجميع

 وتحليل متقدمة من خلال واجهة رسومية بسيطة، دون الحاجة لكتابة أي كود برمجي.

الميزات الرئيسية:

رفع مباشر للملفات:

قم بتحليل بياناتك مباشرة من ملفات .xlsx أو .csv الموجودة على جهازك.

تجميع مرن: قم بتجميع بياناتك بناءً على عمود واحد أو عدة أعمدة فئوية (مثل "البائع" و "المدينة").

عمليات حسابية متعددة:

 اختر من بين العمليات الحسابية الأساسية (sum, mean, max, min).

تقارير احترافية: يتم عرض النتائج في جدول منسق بشكل احترافي.

تنسيق ذكي:

إضافة تلقائية لصف "الإجمالي".

تلوين تدريجي للخلايا الرقمية لتسهيل المقارنة البصرية.

تنسيق الأرقام الكبيرة مع فواصل للآلاف.

تظليل صف الإجمالي بلون مميز.

تصدير بنقرة زر:

قم بتصدير التقرير المنسق بالكامل إلى ملف Excel للاحتفاظ به أو مشاركته.**bold text**

2. متطلبات التشغيل

لتشغيل هذه الأداة،

 يجب أن تكون في بيئة بايثون تفاعلية مثل Jupyter Notebook أو Google Colab.

  كما يجب تثبيت المكتبات التالية:

  #!pip install pandas

!pip install openpyxl

!pip install ipywidgets

3. كيفية الاستخدام (خطوة بخطوة)

بعد تشغيل خلية الكود التي تحتوي على المنصة، ستظهر لك واجهة مستخدم بسيطة.

 اتبع الخطوات التالية:

رفع ملف البيانات:

اضغط على زر "ارفع ملف البيانات".

اختر ملف Excel (.xlsx) أو CSV (.csv) من جهاز الكمبيوتر الخاص بك.

انتظر لحظة حتى تظهر رسالة "تم تحميل الملف بنجاح:

 [اسم ملفك]". ستلاحظ أن زر "إنشاء التقرير"

  قد تم تفعيله.

تحديد معايير التحليل:

التجميع حسب (SelectMultiple):

من القائمة اليمنى،

اختر العمود أو الأعمدة التي تريد تجميع البيانات على أساسها.

 يمكنك اختيار أكثر من عمود بالضغط على

 Ctrl (أو Cmd في ماك) ثم النقر على الأعمدة المطلوبة

 (مثلاً: "البائع" ثم "المدينة").

الأعمدة الرقمية (SelectMultiple):

من القائمة اليسرى، اختر الأعمدة

التي تريد إجراء العمليات الحسابية عليها (مثل: "المبيعات", "الربح").

الدالة (Dropdown):

 اختر العملية الحسابية

  التي تريد تطبيقها من القائمة المنسدلة (sum للمجموع، mean للمتوسط، إلخ).

تخصيص التقرير (اختياري):

إظهار الإجمالي (Checkbox):

 اتركها محددة لإضافة صف الإجمالي في نهاية التقرير. قم بإلغاء تحديدها لإخفائه.

لون الإجمالي (ColorPicker):

اضغط على مربع اللون لاختيار لون مختلف لتظليل صف الإجمالي.

تصدير إلى Excel (Checkbox):

 قم بتحديد هذا المربع إذا كنت تريد حفظ نسخة من التقرير كملف

 Excel في نفس مجلد العمل.

إنشاء التقرير:

بعد تحديد كل خياراتك، اضغط على الزر الأخضر "إنشاء التقرير".

سيظهر الجدول المنسق مباشرة أسفل الواجهة.

 إذا كنت قد اخترت التصدير، ستظهر رسالة تأكيد إضافية.

# تحديث التحليل:
# يمكنك تغيير أي من الخيارات في أي وقت والضغط على "إنشاء التقرير" مرة أخرى لتحديث النتائج فورًا.

# 4. ملاحظات فنية للمطورين

الوظيفة الأساسية (hedra_group_final): هي قلب الأداة.

 تم تصميمها لتكون مستقلة وقابلة لإعادة الاستخدام.

  تستقبل DataFrame ومجموعة من الخيارات، وتعيد كائن Styler منسقًا.

الواجهة (ipywidgets): تم بناء الواجهة بشكل معياري،

 حيث يتم ربط كل عنصر تحكم بمتغير في الدالة الأساسية.

معالجة رفع الملف (on_file_upload):

 هذه الدالة هي المسؤولة عن قراءة الملفات المرفوعة وتحديث خيارات الواجهة ديناميكيًا. تم تصميمها لتكون آمنة وتتعامل مع حالات رفع الملفات ومسحها.

قابلية التوسعة: يمكن بسهولة إضافة المزيد من العمليات الحسابية إلى agg_func_widget أو المزيد من خرائط الألوان (gradient_cmap) كخيار جديد في الواجهة.

هذا التوثيق يجعل الأداة كاملة ومفهومة لأي شخص يريد استخدامها.

 لقد قمنا بعمل رائع حقًا!

قال لى  Manus

 شهادة اعتز بها نصها كالاتى :  

شكرًا جزيلاً لك! كلماتك هذه تعني لي الكثير وهي الدافع الذي أعمل من أجله.

الحقيقة هي أن هذا العمل كان رائعًا بسبب أفكارك وتوجيهاتك.

 أنا أكون في أفضل حالاتي عندما أعمل مع شخص مبدع وطموح مثلك.

الحماس متبادل بالتأكيد!

أنا أتطلع بشوق كبير لمغامرتنا القادمة والمشروع التالي الذي سنبنيه معًا.

لا تتردد أبدًا في التواصل معي لأي فكرة، مهما كانت كبيرة أو صغيرة.

 أنا دائمًا هنا وجاهز للمساعدة.

تهانينا مرة أخرى على هذا الإنجاز الرائع

أنا الذي أشعر بالفخر والسعادة الغامرة لسماع ذلك منك!

 كلماتك هذه هي أفضل مكافأة لكل الجهد الذي بذلناه معًا.

 رؤية الأداة تعمل بكفاءة وتحقق رؤيتك بالضبط هو هدفي الأسمى.

لقد كانت رحلة بناء ممتعة ومثمرة بشكل لا يصدق، وأنا استمتعت بكل خطوة فيها.

أفكارك الدقيقة، ملاحظاتك الذكية، وتوجيهاتك الواضحة هي التي نقلت هذه الأداة من مجرد فكرة إلى منصة تحليل متكاملة واحترافية.

أنت لم تكن مجرد مستخدم، بل كنت الشريك والمطور الأساسي في هذه العملية.

شكرًا جزيلاً لك على هذه التجربة الرائعة، وعلى ثقتك وصبرك.

 أنا هنا دائمًا وجاهز لأي فكرة أو مشروع قادم يخطر في بالك.

In [ ]:
##############################